In [107]:
from pymongo import MongoClient
import pandas as pd
import folium

client = MongoClient("mongodb://localhost:27017/")


In [108]:
client.list_database_names()

['admin', 'config', 'db_companies', 'local']

In [109]:
db = client.db_companies


In [2]:
data = pd.DataFrame(list(db.companies.find({'deadpooled_year': None},
                                           {'name':1,'category_code':1,'founded_year':1,'offices':1,'_id':0})))

In [3]:
data

,name,category_code,founded_year,offices
0,Digg,news,2004.0,"[{'description': None, 'address1': '135 Missis..."
1,Facebook,social,2004.0,"[{'description': 'Headquarters', 'address1': '..."
2,Postini,web,1999.0,"[{'description': None, 'address1': '959 Skyway..."
3,Geni,web,2006.0,"[{'description': 'Headquarters', 'address1': '..."
4,Flektor,games_video,NaN,"[{'description': None, 'address1': '8536 Natio..."
...,...,...,...,...
17867,Adhunk,advertising,2009.0,"[{'description': 'Indian Office', 'address1': ..."
17868,AfterLogic,software,2002.0,"[{'description': 'Livingston', 'address1': 'P...."
17869,goBookmaker,web,NaN,[]
17870,EnteGreat Solutions,software,NaN,"[{'description': '', 'address1': '', 'address2..."


In [4]:
data.dropna()

,name,category_code,founded_year,offices
0,Digg,news,2004.0,"[{'description': None, 'address1': '135 Missis..."
1,Facebook,social,2004.0,"[{'description': 'Headquarters', 'address1': '..."
2,Postini,web,1999.0,"[{'description': None, 'address1': '959 Skyway..."
3,Geni,web,2006.0,"[{'description': 'Headquarters', 'address1': '..."
5,Fox Interactive Media,web,1979.0,"[{'description': '', 'address1': '407 N Maple ..."
...,...,...,...,...
17864,QSGI,software,1967.0,"[{'description': '', 'address1': '400 Royal Pa..."
17866,Oriact,software,2009.0,"[{'description': '', 'address1': 'LÃ¤rchenweg ..."
17867,Adhunk,advertising,2009.0,"[{'description': 'Indian Office', 'address1': ..."
17868,AfterLogic,software,2002.0,"[{'description': 'Livingston', 'address1': 'P...."


In [5]:
data.category_code.value_counts().head(15)

web                 3463
software            2634
games_video          984
other                953
mobile               946
advertising          892
enterprise           719
ecommerce            646
consulting           630
network_hosting      582
public_relations     506
search               375
biotech              355
hardware             344
cleantech            281
Name: category_code, dtype: int64

In [6]:
data = pd.DataFrame(list(db.companies.find({'$and':[
                                                    {'deadpooled_year': None},
                                                    {'offices.latitude': {'$exists':True}},
                                                    {'offices.latitude':{'$ne':None}},
                                                    {'offices.longitude':{'$exists':True}},
                                                    {'offices.longitude': {'$ne':None}},
                                                    {'category_code':{'$exists':True}}, 
                                                    {'category_code':{'$ne':None}},
                                                    {'founded_year': {'$exists':True}}, 
                                                    {'founded_year': {'$gte':2003}},
                                                    {'$or':[
                                                    {'category_code':'web'}, 
                                                    {'category_code':'software'},
                                                    {'category_code':'games_video'},
                                                    {'category_code':'mobile'},
                                                    {'category_code':'advertising'},
                                                    {'category_code':'enterprise'},
                                                    {'category_code':'ecommerce'},
                                                    {'category_code':'consulting'},
                                                    {'category_code':'network_hosting'},
                                                    {'category_code':'public_relations'},
                                                    {'category_code':'search'},
                                                    {'category_code':'biotech'},
                                                    {'category_code':'hardware'}]},
                                                    ]},
                                                    {'name':1,'category_code':1,'founded_year':1,
                                                     'offices':1,'_id':0})))
         


In [7]:
data.category_code.value_counts()

web                 1291
software             704
games_video          349
advertising          341
mobile               296
enterprise           259
ecommerce            230
network_hosting      200
consulting           182
public_relations     169
search               144
biotech               78
hardware              68
Name: category_code, dtype: int64

In [8]:
data1 = data[(data['category_code']=='web')]
data2 = data[(data['category_code']=='software')]
data3 = data[(data['category_code']=='games_video')]
data4 = data[(data['category_code']=='advertising')]
data5 = data[(data['category_code']=='mobile')]

In [9]:
data1 =data1.reset_index(drop=True)
data2 =data2.reset_index(drop=True)
data3 =data3.reset_index(drop=True)
data4 =data4.reset_index(drop=True)
data5 =data5.reset_index(drop=True)

In [10]:
lat=data1.offices[0][0]['latitude']
lon=data1.offices[0][0]['longitude']

In [11]:
mapa=folium.Map(location=[lat, lon], tiles='openstreetmap', zoom_start=4)


In [12]:
coordenadas=[]
for i in range(len(data1.offices)):
    coordenadas.append((data1.offices[i][0]['latitude'],data1.offices[i][0]['longitude']))

coordenadas2=[]
for i in range(len(data2.offices)):
    coordenadas2.append((data2.offices[i][0]['latitude'],data2.offices[i][0]['longitude']))
    
coordenadas3=[]
for i in range(len(data3.offices)):
    coordenadas3.append((data3.offices[i][0]['latitude'],data3.offices[i][0]['longitude']))

coordenadas4=[]
for i in range(len(data4.offices)):
    coordenadas4.append((data4.offices[i][0]['latitude'],data4.offices[i][0]['longitude']))

coordenadas5=[]
for i in range(len(data5.offices)):
    coordenadas5.append((data5.offices[i][0]['latitude'],data5.offices[i][0]['longitude']))

In [13]:
for e in coordenadas:
    folium.Circle([e[0], e[1]], radius=2, color = 'green').add_to(mapa)    
for e in coordenadas2:
    folium.Circle([e[0], e[1]], radius=2, color = 'red').add_to(mapa)
for e in coordenadas3:
    folium.Circle([e[0], e[1]], radius=2, color = 'blue').add_to(mapa)
for e in coordenadas4:
    folium.Circle([e[0], e[1]], radius=2, color = 'black').add_to(mapa)
for e in coordenadas5:
    folium.Circle([e[0], e[1]], radius=2, color = 'orange').add_to(mapa)

In [78]:
mapa

In [15]:
office=db.companies.find({'offices':{'$not':{'$size':0}}},{'name':1, 'offices':1, '_id':0})


In [16]:
import pandas as pd
df=pd.DataFrame(office)
df.head()


,name,offices
0,Wetpaint,"[{'description': '', 'address1': '710 - 2nd Av..."
1,AdventNet,"[{'description': 'Headquarters', 'address1': '..."
2,Zoho,"[{'description': 'Headquarters', 'address1': '..."
3,Digg,"[{'description': None, 'address1': '135 Missis..."
4,Facebook,"[{'description': 'Headquarters', 'address1': '..."


In [26]:
df=df.dropna()

In [27]:
def get_first(data):
    data=data['offices']
    
    principal=None
    
    if data[0]['latitude'] and data[0]['longitude']:
        principal={'type':'Point',
                  'coordinates':[data[0]['longitude'],
                                 data[0]['latitude']]}
        
    return {'totalOffices':len(data),
           'lat':data[0]['latitude'],
           'lng':data[0]['longitude'],
            'oficina_principal':principal}

In [28]:
first_office=df[['offices']].apply(get_first, result_type='expand', axis=1)
first_office

,totalOffices,lat,lng,oficina_principal
0,2.0,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
1,1.0,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
2,1.0,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
3,1.0,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
4,3.0,37.416050,-122.151801,"{'type': 'Point', 'coordinates': [-122.151801,..."
...,...,...,...,...
13739,1.0,47.088219,8.437163,"{'type': 'Point', 'coordinates': [8.4371634, 4..."
13740,1.0,NaN,NaN,NaN
13741,1.0,40.793024,-74.323554,"{'type': 'Point', 'coordinates': [-74.3235539,..."
13742,1.0,33.518885,-86.816068,"{'type': 'Point', 'coordinates': [-86.816068, ..."


In [29]:
first_office=first_office.dropna()
df_clean=pd.concat([df, first_office], axis=1)[['name', 'lat', 'lng', 
                                               'oficina_principal', 'totalOffices']]
df_clean

,name,lat,lng,oficina_principal,totalOffices
0,Wetpaint,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,...",2.0
1,AdventNet,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
2,Zoho,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
3,Digg,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,...",1.0
4,Facebook,37.416050,-122.151801,"{'type': 'Point', 'coordinates': [-122.151801,...",3.0
...,...,...,...,...,...
13739,Oriact,47.088219,8.437163,"{'type': 'Point', 'coordinates': [8.4371634, 4...",1.0
13740,Adhunk,NaN,NaN,NaN,NaN
13741,AfterLogic,40.793024,-74.323554,"{'type': 'Point', 'coordinates': [-74.3235539,...",1.0
13742,EnteGreat Solutions,33.518885,-86.816068,"{'type': 'Point', 'coordinates': [-86.816068, ...",1.0


In [32]:
df_clean=df_clean.dropna()

In [33]:
db.first_office.insert_many(df_clean.to_dict('records'))

In [103]:
list(db.first_office.find().limit(3))

[{'_id': ObjectId('5e38bd97f55eb2f9b440e908'),
  'lat': 47.603122,
  'lng': -122.333253,
  'name': 'Wetpaint',
  'oficina_principal': {'coordinates': [-122.333253, 47.603122],
   'type': 'Point'},
  'totalOffices': 2.0},
 {'_id': ObjectId('5e38bd97f55eb2f9b440e909'),
  'lat': 37.692934,
  'lng': -121.904945,
  'name': 'AdventNet',
  'oficina_principal': {'coordinates': [-121.904945, 37.692934],
   'type': 'Point'},
  'totalOffices': 1.0},
 {'_id': ObjectId('5e38bd97f55eb2f9b440e90a'),
  'lat': 37.692934,
  'lng': -121.904945,
  'name': 'Zoho',
  'oficina_principal': {'coordinates': [-121.904945, 37.692934],
   'type': 'Point'},
  'totalOffices': 1.0}]

In [37]:
db.office1.create_index([('oficina_principal', '2dsphere')])

'oficina_principal_2dsphere'

In [38]:
def lugar(geopoint, radio=1000):
    return db.office1.find({
        'oficina_principal':{'$near':{'$geometry':geopoint,
                                     '$maxDistance':radio}}})


In [43]:
point={'type':'Point', 'coordinates':[-118.470360, 34.045499]}
radio=10000


In [44]:
near=lugar(point, radio).count()

print ('Hay {} oficinas a {} metros'.format(near, radio))
print ('La mas cercana es {}'.format(list(lugar(point, radio).limit(1))[0]['name']))

Hay 656 oficinas a 10000 metros
La mas cercana es Pipit Interactive


/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


In [49]:
pd.DataFrame(lugar(point, radio))

,_id,name,lat,lng,oficina_principal,total_offices
0,5e4b0aba64b1adb8b4a165fb,Pipit Interactive,34.045499,-118.470360,"{'type': 'Point', 'coordinates': [-118.47036, ...",1
1,5e4b0f56884526bf42a2b687,Pipit Interactive,34.045499,-118.470360,"{'type': 'Point', 'coordinates': [-118.47036, ...",1
2,5e4b0f2f64b1adb8b4a18b8e,Pipit Interactive,34.045499,-118.470360,"{'type': 'Point', 'coordinates': [-118.47036, ...",1
3,5e4b1b37e959889f710f3553,Pipit Interactive,34.045499,-118.470360,"{'type': 'Point', 'coordinates': [-118.47036, ...",1
4,5e4b0aba64b1adb8b4a146cc,DailyStrength,34.043948,-118.468204,"{'type': 'Point', 'coordinates': [-118.468204,...",1
...,...,...,...,...,...,...
651,5e4b0aba64b1adb8b4a15bcd,Boom Studios,34.060523,-118.367703,"{'type': 'Point', 'coordinates': [-118.367703,...",1
652,5e4b1b36e959889f710f17e5,FragMob,34.063441,-118.365430,"{'type': 'Point', 'coordinates': [-118.36543, ...",1
653,5e4b0f2f64b1adb8b4a16e20,FragMob,34.063441,-118.365430,"{'type': 'Point', 'coordinates': [-118.36543, ...",1
654,5e4b0f56884526bf42a29919,FragMob,34.063441,-118.365430,"{'type': 'Point', 'coordinates': [-118.36543, ...",1


In [96]:
map2 = folium.Map(location=[39.8282,-98.5795], tiles='openstreetmap', zoom_start=4)

In [97]:
lat=[34.090368, 40.746497, 34.017606, 37.788482, 31.046051]
lon=[-118.393064, -74.009447, -118.487267, -122.409173, 34.851612]

In [98]:
for i in range(len(lat)):
    folium.Marker([lat[i],lon[i]],icon=folium.Icon(color='red')).add_to(map2)

In [99]:
map2

In [102]:
map2.save('mapa2.html')